In [ ]:
# from google.colab import drive
# import os
# drive.mount('/content/drive')
# os.chdir("drive/MyDrive/Bootcamp_Henry/pro_final")
# print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Bootcamp_Henry/pro_final


In [ ]:
# !pip install lightfm -q

# **Objetivo del Cuaderno Jupyter**
- Crear el Modelo de Recomendación basado en el Algoritmo LightFm

Importando Librerias

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from scipy import sparse
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k
import numpy as np
import joblib

Cargando Datos

In [ ]:
reviews = pd.read_parquet('datasets/yelp/ml_tables/ml_reviews.parquet')
df_user = pd.read_parquet('datasets/yelp/ml_tables/ml_user.parquet')
df_business = pd.read_parquet('datasets/yelp/ml_tables/ml_business.parquet')

df_business = df_business.rename(columns={'stars': 'business_stars'})

mask = df_business['business_id'].isin(reviews['business_id'].unique().tolist())
df_business = df_business[mask]

mask = reviews['user_id'].isin(df_user['user_id'].unique().tolist())
reviews = reviews[mask]

## Preprocesamiento Pasando user_id y business_id a enteros

In [ ]:
# Creando la columna de codigos enteros en `user`
df_user['user_id_int'] = range(1, len(df_user) + 1)
cols = ['user_id_int'] + [col for col in df_user.columns if col != 'user_id_int']
df_user = df_user[cols]
df_user_ids = df_user[['user_id_int', 'user_id']]
user = df_user[['user_id_int', 'average_stars']]

# Creando la columna de codigos enteros en `business`
df_business['business_id_int'] = range(1, len(df_business) + 1)
cols = ['business_id_int'] + [col for col in df_business.columns if col != 'business_id_int']
df_business = df_business[cols]
df_business_ids = df_business[['business_id_int', 'business_id']]

# Creando la columna de codigos enteros en `reviews
reviews = reviews.merge(df_user_ids, on='user_id', how='left')
reviews = reviews.merge(df_business_ids, on='business_id', how='left')
reviews = reviews[['user_id_int', 'business_id_int', 'stars']]

In [1]:
# Preprocesamiento
numerical_cols = ['business_stars', 'review_count', 'cantidad_checkins', 'compliment_count']  # Lista de columnas numéricas
scaler = StandardScaler()
df_business[numerical_cols] = scaler.fit_transform(df_business[numerical_cols])

user['average_stars'] = scaler.fit_transform(user[['average_stars']])

business = df_business[['business_id_int', 'business_stars', 'polaridad_tip']]

In [ ]:
# Crear el dataset de LightFM
dataset = Dataset()
dataset.fit(
    (x.user_id_int for x in reviews.itertuples()),
    (x.business_id_int for x in reviews.itertuples()),
    user_features=(col for col in user.columns if col != 'user_id_int'),
    item_features=(col for col in business.columns if col != 'business_id_int')
)

In [ ]:
# Construir Matriz de Interacciones
(interactions, weights) = dataset.build_interactions(
    [(x['user_id_int'], x['business_id_int']) for _, x in reviews.iterrows()]
)

In [ ]:
dataset.fit_partial(item_features=((lambda column: x[column] for column in business.columns[1:]) for x in business.iterrows()))

In [ ]:
dataset.fit_partial(user_features=((lambda column: x[column] for column in user.columns[1:]) for x in user.iterrows()))

In [ ]:
# Construir Matriz de Características de Usuarios
user_features = dataset.build_user_features(
    [(x['user_id_int'], {col: x[col] for col in user.columns if col != 'user_id_int'}) for _, x in user.iterrows()]
)

In [ ]:
# Construir Matriz de Características de Ítems
item_features = dataset.build_item_features(
    [(x['business_id_int'], {col: x[col] for col in business.columns if col not in ['business_id_int', 'state']}) for _, x in business.iterrows()]
)

In [ ]:
# Inicializar el modelo
model = LightFM(loss='warp', item_alpha=1e-10, user_alpha=1e-10, random_state=42)

In [ ]:
# Entrenar el modelo
model.fit(interactions, item_features=item_features, user_features=user_features, epochs=400, num_threads=5)

In [ ]:
# Evaluar el modelo
train_precision = precision_at_k(model, interactions, item_features=item_features, user_features=user_features, k=5).mean()
print(f'Train precision at k=5: {train_precision}')

Train precision at k=5: 0.14722798764705658


In [ ]:
"""
 `precision_at_k` se refiere a la proporción de elementos relevantes (es decir, aquellos que el usuario realmente interactuó)
  entre los primeros ( k ) elementos recomendados.

  train_precision = precision_at_k(model, interactions, item_features=item_features, user_features=user_features, k=5).mean()

  Trataremos de ajustar esa proporcion lo mas posible, pero hay que tener en cuenta que en los sistemas de recomendacion con poca data
  y sobre todo pocas interacciones de los usuarios (usuarios con un una o muy pocas reseñas) este valor suele ser bajo.

  Nota: En este Caso estamos sacando el Promedio con respecto a las primeras 5 recomendaciones. Si tomaramos el promedio de todas las primeras
        recomendaciones a los usuarios solamente (es decir k=1) el numero fuera aproximadamente 3 veces mayor
"""

alphas = [1e-8, 1e-9, 1e-10]
threads = [2,3,4,5,6]
epochs = [180, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400]

parametros = []
for i in range(len(alphas)):
  for j in range(len(threads)):
    for k in range(len(epochs)):
      diccionario = {}
      diccionario['alpha'] = alphas[i]
      diccionario['threads'] = threads[j]
      diccionario['epochs'] = epochs[k]
      model = LightFM(loss='warp',item_alpha=alphas[i], user_alpha=alphas[i], random_state=42)
      model.fit(interactions, item_features=item_features, user_features=user_features, epochs=epochs[k], num_threads=threads[j])
      train_precision = precision_at_k(model, interactions, item_features=item_features, user_features=user_features, k=5).mean()
      diccionario['train_precision'] = train_precision
      parametros.append(diccionario)
      print(f'allphas --> {alphas[i]}   threads --> {threads[j]}   epochs --> {epochs[k]}   train_precision --> {train_precision}')

df_parametros = pd.DataFrame(parametros)
df_parametros.to_csv('parametros.csv')


allphas --> 1e-08   threads --> 2   epochs --> 180   train_precision --> 0.07833496481180191
allphas --> 1e-08   threads --> 2   epochs --> 200   train_precision --> 0.08227275311946869
allphas --> 1e-08   threads --> 2   epochs --> 210   train_precision --> 0.0874723568558693
allphas --> 1e-08   threads --> 2   epochs --> 220   train_precision --> 0.09167482703924179
allphas --> 1e-08   threads --> 2   epochs --> 230   train_precision --> 0.09681279957294464
allphas --> 1e-08   threads --> 2   epochs --> 240   train_precision --> 0.09642119705677032
allphas --> 1e-08   threads --> 2   epochs --> 250   train_precision --> 0.10311832278966904
allphas --> 1e-08   threads --> 2   epochs --> 260   train_precision --> 0.10547518730163574
allphas --> 1e-08   threads --> 2   epochs --> 270   train_precision --> 0.10868777334690094
allphas --> 1e-08   threads --> 2   epochs --> 280   train_precision --> 0.10745133459568024
allphas --> 1e-08   threads --> 2   epochs --> 290   train_precision --

In [ ]:
"""Completando desde donde se interrumpio el proceso de la celda pasada

Nota: Tomar en cuenta que aunque sembramos una semilla con `random_state==42` al entrenar el algoritmo nuevamente nos 
      va a dar un valor muy aproximado de precision_at_k pero dificilmente nos de el mismo, esto se debe principalmente
      a 2 Factores, el primero es que LightFm introduce como metodo de optimización el descenso del gradiente 
      estacástico, que es una variación que introduce cierta aleatoriedad en el proceso.  La segunda Razon es que 
      estamos utilizando varios Hilos de ejecusión, y al ejecutar en paralelo varios procesos siempre el orden como 
      se vuelve a juntar todo puede variar. """
alphas = [1e-10]
threads = [5,6]
epochs = [180, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400]

parametros = []
for i in range(len(alphas)):
  for j in range(len(threads)):
    for k in range(len(epochs)):
      diccionario = {}
      diccionario['alpha'] = alphas[i]
      diccionario['threads'] = threads[j]
      diccionario['epochs'] = epochs[k]
      model = LightFM(loss='warp',item_alpha=alphas[i], user_alpha=alphas[i], random_state=42)
      model.fit(interactions, item_features=item_features, user_features=user_features, epochs=epochs[k], num_threads=threads[j])
      train_precision = precision_at_k(model, interactions, item_features=item_features, user_features=user_features, k=5).mean()
      diccionario['train_precision'] = train_precision
      parametros.append(diccionario)
      print(f'allphas --> {alphas[i]}   threads --> {threads[j]}   epochs --> {epochs[k]}   train_precision --> {train_precision}')

df_parametros = pd.DataFrame(parametros)
df_parametros.to_csv('parametros.csv')

allphas --> 1e-10   threads --> 5   epochs --> 180   train_precision --> 0.07149642705917358
allphas --> 1e-10   threads --> 5   epochs --> 200   train_precision --> 0.07687371224164963
allphas --> 1e-10   threads --> 5   epochs --> 210   train_precision --> 0.0919395238161087
allphas --> 1e-10   threads --> 5   epochs --> 220   train_precision --> 0.08997426182031631
allphas --> 1e-10   threads --> 5   epochs --> 230   train_precision --> 0.09997100383043289
allphas --> 1e-10   threads --> 5   epochs --> 240   train_precision --> 0.10029371082782745
allphas --> 1e-10   threads --> 5   epochs --> 250   train_precision --> 0.097621388733387
allphas --> 1e-10   threads --> 5   epochs --> 260   train_precision --> 0.10576888918876648
allphas --> 1e-10   threads --> 5   epochs --> 270   train_precision --> 0.10932594537734985
allphas --> 1e-10   threads --> 5   epochs --> 280   train_precision --> 0.10771603137254715
allphas --> 1e-10   threads --> 5   epochs --> 290   train_precision --> 

In [ ]:
"""Aca simplemente se observan los parámetros de los que tuvieron mejor train_precision
Nota: Antes de hacer esta corrida se evaluó de forma manual el modelo con distintas combinaciones de Variables 
      predictoras aca solamente hicimos una corrida con las variables predictoras que nos venían arrojando mejores 
      resultados.
Nota: Para el 3cer Sprint con la incorporación de mas datos TODO este proceso hay que repetirlo y estudiar nuevamente 
      la conveniecia de distintas combinaciones de las variables predictoras """
df_parametros.sort_values(by='train_precision', ascending=False).head()

,alpha,threads,epochs,train_precision
21,1.000000e-10,5,400,0.137159
41,1.000000e-10,6,380,0.135335
43,1.000000e-10,6,400,0.133272
40,1.000000e-10,6,370,0.133065
18,1.000000e-10,5,370,0.133040


Creando la Función que devuelve las recomendaciones

Con pequeños ajustes esta sería la función que nos llevaremos a la Api para responder las peticiones.

In [ ]:
# Función para recomendar restaurantes
def recomendar_restaurantes(model:LightFM, user_id_string:str, state:str, df_user_ids:pd.DataFrame,
                            df_business:pd.DataFrame, item_features:sparse.csr_matrix,
                            user_features:sparse.csr_matrix, top_n:int=5) -> list[dict]|str:
    # Obtener el ID Entero del usuario
  try:
    mask = df_user_ids['user_id'] == user_id_string
    user_id = int(df_user_ids[mask]['user_id_int'].values[0])
  except:
    return 'Usuario no encontrado'
  # Creando la lista de Negocios Segun el estado
  mask = df_business['state'] == state
  business_id_list = df_business[mask]['business_id_int'].tolist()

  # Aplicando procedimiento estandard de LightFm para obtener Recomendaciones
  scores = model.predict(user_id, business_id_list, item_features=item_features, user_features=user_features)
  top_items = np.argsort(-scores)[:top_n]
  recomendations = [business_id_list[i] for i in top_items]

  # Obteniendo las Caracteristicas de cada Recomendación
  recomendations_list = []
  for recomendation in recomendations:
    recomendation_dict = {}
    mask = df_business['business_id_int'] == recomendation
    recomendation_dict['business_id'] = df_business[mask]['business_id'].values[0]
    recomendation_dict['name'] = df_business[mask]['name'].values[0]
    recomendation_dict['address'] = df_business[mask]['address'].values[0]
    recomendation_dict['city'] = df_business[mask]['city'].values[0]
    recomendation_dict['latitude'] = df_business[mask]['latitude'].values[0]
    recomendation_dict['longitude'] = df_business[mask]['longitude'].values[0]
    recomendations_list.append(recomendation_dict)

  return recomendations_list

Probando como Funcionaría la Función de Recomendaciones de Implementarse en una App o en una Api

In [ ]:
# Ejemplo de uso
user_id_string = 'A-urIi0zR2sAcnN3VccI5Q'
# user_id_string = 'aaaaaaaaaaaaaaaaaaaaaaa'
state = 'CA'

recomendations_list = recomendar_restaurantes(model=model, user_id_string=user_id_string, state=state, df_user_ids=df_user_ids,
                                              df_business=df_business, item_features=item_features, user_features=user_features)
recomendations_list

[{'business_id': 'gaNQrd95h2-4TZaUDPTAIQ',
  'name': 'Rooted Vine Tours',
  'address': '6 Harbor Way, Ste 127',
  'city': 'Santa Barbara',
  'latitude': 34.4037461,
  'longitude': -119.6930334},
 {'business_id': 'tzR6CHZNmnXzADX-SoM2sQ',
  'name': 'Grapeline Wine Tours',
  'address': '',
  'city': 'Santa Barbara',
  'latitude': 34.4208305,
  'longitude': -119.6981901},
 {'business_id': '5RsVAkDnMrcSbErS6P1eew',
  'name': 'Cajé Coffee Roasters - Isla Vista',
  'address': '948 Embarcadero Del Norte',
  'city': 'Isla Vista',
  'latitude': 34.411997,
  'longitude': -119.855139},
 {'business_id': 'NORF647ibBQ4U-VNMgzSLg',
  'name': 'Oakberry Açaí Bowls and Smoothies',
  'address': '819-B State St',
  'city': 'Santa Barbara',
  'latitude': 34.4196823736,
  'longitude': -119.7003016706},
 {'business_id': 'XkveGZ8pr9TL5bZCqmzuhA',
  'name': 'Hustle & Grind Coffee Company',
  'address': '11 W De La Guerra St',
  'city': 'Santa Barbara',
  'latitude': 34.4191062,
  'longitude': -119.6995489}]

Guardando TODO lo necesario para Implementar el Modelo

In [ ]:
# Guardando el Modelo
joblib.dump(model, 'model/model.pkl')

# Guardando las matrices de características
joblib.dump(user_features, 'model/user_features.pkl')
joblib.dump(item_features, 'model/item_features.pkl')

# Guardando Dataframes necesarios para la Colaboración en el Filtrado
df_user_ids.to_parquet('model/df_user_ids.parquet')
df_business.to_parquet('model/df_business.parquet')


Como sería la carga de la Información

In [ ]:
# Cargando el Modelo
model = joblib.load('model/model.pkl')

# Cargando las matrices
user_features = joblib.load('model/user_features.pkl')
item_features = joblib.load('model/item_features.pkl')

# Cargando los DataFrames
df_user_ids = pd.read_parquet('model/df_user_ids.parquet')
df_business = pd.read_parquet('model/df_business.parquet')